In [1]:
# 출력을 원하실 경우 print() 함수 활용
# 예시) print(df.head())

# getcwd(), chdir() 등 작업 폴더 설정 불필요
# 파일 경로 상 내부 드라이브 경로(C: 등) 접근 불가

# 데이터 파일 읽기 예제
import pandas as pd
import numpy as np
X_test = pd.read_csv("yemoonsaBigdata/datasets/Part3/304_x_test.csv")
X_train = pd.read_csv("yemoonsaBigdata/datasets/Part3/304_x_train.csv")
y_train = pd.read_csv("yemoonsaBigdata/datasets/Part3/304_y_train.csv")

# 사용자 코딩

# 답안 제출 참고
# 아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용
# pd.DataFrame({'index': X_test.index, 'y_pred': pred}).to_csv('003000000.csv', index=False)

## 1.데이터 전처리

In [2]:
#결측치 확인 
print(X_train.info())
print(X_train.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   1490 non-null   int64 
 1   Age                  1490 non-null   int64 
 2   Employment Type      1490 non-null   object
 3   GraduateOrNot        1490 non-null   object
 4   AnnualIncome         1490 non-null   int64 
 5   FamilyMembers        1490 non-null   int64 
 6   ChronicDiseases      1490 non-null   int64 
 7   FrequentFlyer        1490 non-null   object
 8   EverTravelledAbroad  1490 non-null   object
dtypes: int64(5), object(4)
memory usage: 104.9+ KB
None
                ID          Age  AnnualIncome  FamilyMembers  ChronicDiseases
count  1490.000000  1490.000000  1.490000e+03    1490.000000      1490.000000
mean    998.832886    29.668456  9.386577e+05       4.732215         0.285235
std     573.158458     2.935416  3.731487e+05       1.612608    

In [3]:
print(X_train)
print(y_train)

        ID  Age               Employment Type GraduateOrNot  AnnualIncome  \
0     1704   26  Private Sector/Self Employed           Yes       1400000   
1      491   28  Private Sector/Self Employed           Yes       1100000   
2      414   33  Private Sector/Self Employed           Yes       1400000   
3      120   28  Private Sector/Self Employed           Yes        800000   
4     1268   33             Government Sector           Yes       1000000   
...    ...  ...                           ...           ...           ...   
1485  1045   28  Private Sector/Self Employed           Yes       1100000   
1486   342   28  Private Sector/Self Employed           Yes       1100000   
1487   393   29  Private Sector/Self Employed           Yes       1050000   
1488   572   28  Private Sector/Self Employed            No        550000   
1489   658   30  Private Sector/Self Employed            No       1450000   

      FamilyMembers  ChronicDiseases FrequentFlyer EverTravelledAbroad  
0 

In [4]:
y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   ID               1490 non-null   int64
 1   TravelInsurance  1490 non-null   int64
dtypes: int64(2)
memory usage: 23.4 KB


In [5]:
#변수구분
COL_DEL = ['ID']
COL_NUM = ['Age','AnnualIncome','FamilyMembers']
COL_CAT = ['Employment Type','GraduateOrNot','ChronicDiseases','FrequentFlyer','EverTravelledAbroad']
COL_Y = ['TravelInsurance'] 

In [6]:
y_train[COL_Y].values.ravel()

array([0, 1, 1, ..., 0, 0, 1])

In [7]:
y_train[COL_Y]

,TravelInsurance
0,0
1,1
2,1
3,0
4,1
...,...
1485,0
1486,1
1487,0
1488,0


In [8]:
#학습 검증 데이터분리
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train[COL_NUM+COL_CAT],
                                            y_train[COL_Y],
                                            test_size=0.3)

In [9]:
#정규화
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
std.fit(X_train[COL_NUM])

X_tr[COL_NUM]=std.transform(X_tr[COL_NUM])
X_val[COL_NUM]=std.transform(X_val[COL_NUM])
X_test[COL_NUM]=std.transform(X_test[COL_NUM])

In [10]:
#범주화
from sklearn.preprocessing import LabelEncoder

for col in COL_CAT:
    le =LabelEncoder()
    le.fit(X_train[col])

    X_tr[col] = le.transform(X_tr[col])
    X_val[col] = le.transform(X_val[col])
    X_test[col] = le.transform(X_test[col])

    print(col)
    print(le.classes_)

Employment Type
['Government Sector' 'Private Sector/Self Employed']
GraduateOrNot
['No' 'Yes']
ChronicDiseases
[0 1]
FrequentFlyer
['No' 'Yes']
EverTravelledAbroad
['No' 'Yes']


## 3.모델생성

In [11]:
from sklearn.ensemble import RandomForestClassifier
modelRF=RandomForestClassifier(random_state=123)
modelRF.fit(X_tr,y_tr[COL_Y])

from xgboost import XGBClassifier
modelXG = XGBClassifier(random_state=123)
modelXG.fit(X_tr,y_tr[COL_Y])

/var/folders/cb/0cgtts7x7ps8zly4sl8xwntc0000gn/T/ipykernel_50372/4168522439.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  modelRF.fit(X_tr,y_tr[COL_Y])


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=123, ...)

In [12]:
y_val_predRF = modelRF.predict(X_val)
y_val_predXG = modelXG.predict(X_val)

In [13]:
from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_val,y_val_predRF))
print(roc_auc_score(y_val,y_val_predXG))

0.7371792102307093
0.7543312019355396


In [14]:
def get_score(model,X_tr,X_val, y_tr, y_val):
    y_pred_tr = model.predict(X_tr)
    y_pred_val = model.predict(X_val)
    tr_score = roc_auc_score(y_tr, y_pred_tr)
    val_score = roc_auc_score(y_val, y_pred_val)
    return tr_score,val_score

In [45]:
tr_score,val_score = get_score(modelRF,X_tr,X_val, y_tr, y_val)
print(f'model RF -tr score : {tr_score}, -val score {val_score}')
tr_score,val_score = get_score(modelXG,X_tr,X_val, y_tr, y_val)
print(f'model XG -tr score : {tr_score}, -val score {val_score}')


model RF -tr score : 0.9367687148681575, -val score 0.7356661581489956
model XG -tr score : 0.9104102836280588, -val score 0.7397025171624714


In [53]:
y_pred = modelXG.predict(X_test[COL_NUM+COL_CAT])
result = pd.DataFrame({'ID': X_test['ID'], 'TravelInsurance' : y_pred})
result

,ID,TravelInsurance
0,1569,0
1,1344,1
2,1429,0
3,896,0
4,101,1
...,...,...
492,1376,0
493,87,0
494,287,0
495,337,0
